In [1]:
import os
import math
import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import datasets
from datasets import load_dataset

C:\Users\gguaquiere\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [3]:
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# **Data**

In [5]:
def load_train_test_val_data(name_dataset : str):

    data = load_dataset('silicone',name_dataset)
    train = pd.DataFrame(data['train'])
    test = pd.DataFrame(data['test'])
    val = pd.DataFrame(data['validation'])

    return train, test, val

In [6]:
dyda_da_train, dyda_da_test, dyda_da_val = load_train_test_val_data("dyda_e")

Reusing dataset silicone (C:\Users\gguaquiere\.cache\huggingface\datasets\silicone\dyda_e\1.0.0\af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)
100%|██████████| 3/3 [00:00<00:00, 187.49it/s]


In [7]:
dyda_da_train

,Utterance,Emotion,Dialogue_ID,Label,Idx
0,"say , jim , how about going for a few beers af...",no emotion,1,4,0
1,you know that is tempting but is really not go...,no emotion,1,4,1
2,what do you mean ? it will help us to relax .,no emotion,1,4,2
3,do you really think so ? i don't . it will jus...,no emotion,1,4,3
4,i guess you are right.but what shall we do ? i...,no emotion,1,4,4
...,...,...,...,...,...
87165,i want a pair of locus .,no emotion,11117,4,87165
87166,"take a look at the ones on display , please .",no emotion,11117,4,87166
87167,i need size 41 .,no emotion,11117,4,87167
87168,"could i have the check , please ?",no emotion,11118,4,87168


In [8]:
#on réduit la taille du dataset pour avoir un temps d'entraînement raisonnable

train = dyda_da_train[['Utterance','Emotion']].sample(frac=0.05)
test = dyda_da_test[['Utterance','Emotion']].sample(frac=0.05)
val = dyda_da_val[['Utterance','Emotion']].sample(frac=0.06)

In [9]:
train.Emotion.value_counts()

no emotion    3556
happiness      613
surprise        68
sadness         52
anger           48
disgust         15
fear             6
Name: Emotion, dtype: int64

In [10]:
os.makedirs("model", exist_ok=True)
bert_model_name="uncased_L-12_H-768_A-12"
bert_ckpt_dir = 'model/uncased_L-12_H-768_A-12'
bert_ckpt_file = 'model/uncased_L-12_H-768_A-12/bert_model.ckpt'
bert_config_file = 'model/uncased_L-12_H-768_A-12/bert_config.json'

# **Input Text Preparation**

In [11]:
class DataPreparation:
    
    text_column = "Utterance"
    label_column = "Emotion"

    def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes

        ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self.prepare_data, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(self.data_padding, [self.train_x, self.test_x])

    def prepare_data(self, df):
        x, y = [], []

        for _, row in tqdm(df.iterrows()):
            text, label = row[DataPreparation.text_column], row[DataPreparation.label_column]
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            x.append(token_ids)
            y.append(self.classes.index(label))

        return np.array(x), np.array(y)

    def data_padding(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)

In [12]:
tokenizer = FullTokenizer(vocab_file=bert_ckpt_dir+"/vocab.txt")

# **Model**

In [13]:
def model_defination(max_seq_len, bert_ckpt_file):
    
    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
        
    input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
    bert_output = bert(input_ids)

    print("bert shape", bert_output.shape)

    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    load_stock_weights(bert, bert_ckpt_file)

    return model

In [14]:
classes = train.Emotion.unique().tolist()

data = DataPreparation(train, test, tokenizer, classes, max_seq_len=128)

4358it [00:01, 3898.02it/s]
C:\Users\gguaquiere\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
387it [00:00, 4042.64it/s]


max seq_len 156


In [15]:
model = model_defination(data.max_seq_len, bert_ckpt_file)

bert shape (None, 128, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x000001A933252860> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 128)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 128, 768)          108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 5383  

In [17]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [ ]:
history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=3,
  #callbacks=[tensorboard_callback]
)

Train on 3922 samples, validate on 436 samples
Epoch 1/3
3922/3922 [==============================] - 2406s 613ms/sample - loss: 1.3608 - acc: 0.8078 - val_loss: 1.3513 - val_acc: 0.8142
Epoch 2/3
3696/3922 [===========================>..] - ETA: 2:24 - loss: 1.3519 - acc: 0.8136